## Use the pre-trained model to generate embeddings directly

In [1]:
import os
import numpy as np
import librosa
import tensorflow as tf

In [2]:
from math import floor
from lib_augment import load_and_normalize
from lib_feature import feature_whistleness

%matplotlib inline
import matplotlib.pyplot as plt


In [3]:
# deploy_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald/STAR2000_48kHz'
species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}
fs = 48000
time_reso = 0.02
hop_length = int(time_reso*fs)  #
species_list = list(species_dict.keys())
species_id = list(species_dict.values())

In [4]:
species_to_code = {'bottlenose': 'BD', 'longbeaked_common': 'CD', 'shortbeaked_common': 'CD', 'common': 'CD',
                   'striped': 'STR', 'spotted': 'SPT', 'spinner': 'SPIN', 'pilot': 'PLT', 'roughtoothed': 'RT',
                   'false_killer': 'FKW'}

In [5]:
# species_dict = {'BD': 0, 'CD': 1, 'STR': 2, 'SPT': 3, 'SPIN': 4, 'PLT': 5, 'RT': 6,  'FKW': 7}

step_sec = 1.0  # sec
whistle_ratio = 0.2  # percentage of whislteness frames in a long window
# step_size = int(1.0*fs)
win_long_sec = 10   # 10 sec
step_ind = int(win_long_sec/step_sec)
step_size = int(step_sec*fs)
win_size = int(2.0*fs)

In [6]:
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'

In [7]:
# loading the trained model
from tensorflow.keras.models import load_model
import tensorflow_addons.layers.spatial_pyramid_pooling as spp


model_whistleness_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_whistleness/2021-02-02_200514_resnet18_expt_alldata_run0_f1_lr_0.001/epoch_163_valloss_0.2434_valacc_0.9472.hdf5'
# model_whistleness = load_model(model_whistleness_path)
# model_whistleness.summary()

# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210303_233728/STAR2000/epoch_53_valloss_0.9505_valacc_0.7552.hdf5'
# model_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210305_105231/STAR2000/epoch_56_valloss_0.1945_valacc_0.7305.hdf5'
model_species_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__fit_result_species/20210310_095616/STAR2000/epoch_48_valloss_0.1938_valacc_0.7403.hdf5'

model_species = load_model(model_species_path)
model_species.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 100, 128, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 128, 16)      160       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 128, 16)      64        
_________________________________________________________________
activation (Activation)      (None, 100, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 128, 16)      2320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 128, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 100, 128, 16)      0     

In [8]:
sound_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__sound_48k/__whistle_oswald'
sound_deploy = ['STAR2000_48kHz', 'STAR2003_48kHz', 'STAR2006_48kHz', 'HICEAS2002_48kHz', 'PICEAS2005_48kHz']

## Loading features for STAR 2000

In [9]:
# np.savez(os.path.join(ten_sec_path, sound_deploy[dd]), feature_train=feature_train, label_train=label_train, feature_test=feature_test, label_test=label_test)
feature_curr = np.load(os.path.join(ten_sec_path, 'STAR2000_48kHz_test_others_train.npz'))
feature_train = feature_curr['feature_train']
feature_test = feature_curr['feature_test']
label_train = feature_curr['label_train']
label_test = feature_curr['label_test']
del feature_curr

## Loading trained model

In [13]:
model_species.layers[53].name

'dense_3'

In [11]:
model_species.layers[-9]

In [12]:
model_species.trainable = False

In [14]:
model_feature = tf.keras.Sequential()
for layer in model_species.layers[0:-9]:
    print(layer.name)
    model_feature.add(layer)
input_shape = (None, 100, 128, 1)
model_feature.build(input_shape)
print(model_feature.summary())

input
conv2d
batch_normalization
activation
conv2d_1
batch_normalization_1
activation_1
max_pooling2d
dropout
conv2d_2
batch_normalization_2
activation_2
conv2d_3
batch_normalization_3
activation_3
max_pooling2d_1
dropout_1
conv2d_4
batch_normalization_4
activation_4
conv2d_5
batch_normalization_5
activation_5
max_pooling2d_2
dropout_2
conv2d_6
batch_normalization_6
activation_6
conv2d_7
batch_normalization_7
activation_7
max_pooling2d_3
dropout_3
conv2d_8
batch_normalization_8
activation_8
conv2d_9
batch_normalization_9
activation_9
spatial_pyramid_pooling2d
reshape
dense
batch_normalization_10
activation_10
dropout_4
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 100, 128, 16)      160       
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 128, 16)      64        
_______________________

In [24]:
def spec_to_embedding(model, feature_target):
    embedding_target = []
    num_target = feature_target.shape[0]
    for ii in range(num_target):
        if ii % 100 == 0:
            print(ii)
        feature_long = feature_target[ii, :, :, :, :]
        #  print(feature_long.shape)

        embedding_target.append(model.predict(feature_long))
    embedding_target = np.stack(embedding_target)
    return embedding_target

In [25]:
embedding_test = spec_to_embedding(model_feature, feature_test)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500


In [26]:
embedding_train = spec_to_embedding(model_feature, feature_train)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [28]:
dd = 0
ten_sec_path = '/home/ys587/__Data/__whistle/__whislte_30_species/__dataset/ten_seconds_data'
np.savez(os.path.join(ten_sec_path, sound_deploy[dd]+'_test_others_train_embedding'), embedding_train=embedding_train, label_train=label_train, embedding_test=embedding_test, label_test=label_test)

In [18]:
# embedding_train = []
# num_train = feature_train.shape[0]
# for ii in range(num_train):
#     feature_long = feature_train[ii, :, :, :, :]
#     print(feature_long.shape)
    
#     embedding_train.append(model_feature.predict(feature_long))


(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(10, 256)
(10, 100, 128, 1)
(1

KeyboardInterrupt: 